<a href="https://colab.research.google.com/github/JIUEntong/Customer_Churn_Analysis/blob/master/Customer_Churn_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Customer Churn Prediction Project


1.   Project Goal:

      In this project, we use supervised machine learning models to predict customer churn. Also, we will analyze top factors that influence user retention.
2.   Data Source:[Telco Customer Churn Dataset From Kaggle](https://www.kaggle.com/blastchar/telco-customer-churn)




#Contents


<li>[Part 1: Data Exploration and Visuilization]
<li>[Part 2: Feature Preprocessing]
<li>[Part 3: Model Training and Results Evaluation]
<li>[Part 4: Feature Selection]




#Part 1:Data Exploration and Visuilization

## Part1.1  Load Data and Understand the Raw Dataset

In [2]:
from __future__ import division # for python2, python 2 default integer division

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
url = 'https://raw.githubusercontent.com/JIUEntong/Customer_Churn_Analysis/master/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(url)
df.shape

(7043, 21)

In [3]:
df.dtypes # column data type

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
#change TotalCharges's data type
#df['TotalCharges'] = df['TotalCharges'].astype(float)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')
df.dtypes


customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [5]:
df.head()

customerID  gender  SeniorCitizen  ... MonthlyCharges TotalCharges  Churn
0  7590-VHVEG  Female              0  ...          29.85        29.85     No
1  5575-GNVDE    Male              0  ...          56.95      1889.50     No
2  3668-QPYBK    Male              0  ...          53.85       108.15    Yes
3  7795-CFOCW    Male              0  ...          42.30      1840.75     No
4  9237-HQITU  Female              0  ...          70.70       151.65    Yes

[5 rows x 21 columns]

In [6]:
df.tail()

customerID  gender  SeniorCitizen  ... MonthlyCharges TotalCharges  Churn
7038  6840-RESVB    Male              0  ...          84.80      1990.50     No
7039  2234-XADUH  Female              0  ...         103.20      7362.90     No
7040  4801-JZAZL  Female              0  ...          29.60       346.45     No
7041  8361-LTMKD    Male              1  ...          74.40       306.60    Yes
7042  3186-AJIEK    Male              0  ...         105.65      6844.50     No

[5 rows x 21 columns]

In [7]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [0]:
df = df.dropna()

## Part1.2  Use pandas-profiling to have a basic understanding of dataset.

In [9]:
import pandas_profiling
pandas_profiling.ProfileReport(df)



# Part 2: Feature Preprocessing


In [11]:
for item in df.columns:
    print(item)
    print (df[item].unique())

customerID
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
gender
['Female' 'Male']
SeniorCitizen
[0 1]
Partner
['Yes' 'No']
Dependents
['No' 'Yes']
tenure
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService
['No' 'Yes']
MultipleLines
['No phone service' 'No' 'Yes']
InternetService
['DSL' 'Fiber optic' 'No']
OnlineSecurity
['No' 'Yes' 'No internet service']
OnlineBackup
['Yes' 'No' 'No internet service']
DeviceProtection
['No' 'Yes' 'No internet service']
TechSupport
['No' 'Yes' 'No internet service']
StreamingTV
['No' 'Yes' 'No internet service']
StreamingMovies
['No' 'Yes' 'No internet service']
Contract
['Month-to-month' 'One year' 'Two year']
PaperlessBilling
['Yes' 'No']
PaymentMethod
['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (autom

In [0]:
# Change yes or no to boolean 
df["Churn"].replace(to_replace='Yes', value=1, inplace=True)
df["Churn"].replace(to_replace='No', value=0, inplace=True)
y = df["Churn"]
# Drop y in Xs
to_drop = ["Churn","customerID"]
churn_feat_space = df.drop(to_drop, axis=1)

# yes and no have to be converted to boolean values
yes_no_cols = ["Partner","Dependents","PhoneService","PaperlessBilling","Partner","PhoneService"]
churn_feat_space[yes_no_cols] = churn_feat_space[yes_no_cols] == 'Yes'

churn_feat_space = pd.get_dummies(churn_feat_space)
X = churn_feat_space



In [13]:
print (y.head())

for item in X.columns:
    print(item)
    print (X[item].unique())

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64
SeniorCitizen
[0 1]
Partner
[ True False]
Dependents
[False  True]
tenure
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService
[False  True]
PaperlessBilling
[ True False]
MonthlyCharges
[29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges
[  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
gender_Female
[1 0]
gender_Male
[0 1]
MultipleLines_No
[0 1]
MultipleLines_No phone service
[1 0]
MultipleLines_Yes
[0 1]
InternetService_DSL
[1 0]
InternetService_Fiber optic
[0 1]
InternetService_No
[0 1]
OnlineSecurity_No
[1 0]
OnlineSecurity_No internet service
[0 1]
OnlineSecurity_Yes
[0 1]
OnlineBackup_No
[0 1]
OnlineBackup_No internet service
[0 1]
OnlineBackup_Yes
[1 0]
DeviceProtection_No
[1 0]
DeviceProtection_No internet service
[0 1]
DeviceProtect

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

X

array([[-0.44032709,  1.03561683, -0.65230493, ..., -0.5253508 ,
         1.40476387, -0.54360352],
       [-0.44032709, -0.9656081 , -0.65230493, ..., -0.5253508 ,
        -0.71186341,  1.83957601],
       [-0.44032709, -0.9656081 , -0.65230493, ..., -0.5253508 ,
        -0.71186341,  1.83957601],
       ...,
       [-0.44032709,  1.03561683,  1.53302536, ..., -0.5253508 ,
         1.40476387, -0.54360352],
       [ 2.27103902,  1.03561683, -0.65230493, ..., -0.5253508 ,
        -0.71186341,  1.83957601],
       [-0.44032709, -0.9656081 , -0.65230493, ..., -0.5253508 ,
        -0.71186341, -0.54360352]])

In [15]:
y.unique()

array([0, 1])

#Part 3: Model Training and Results Evaluation


## 3.1 Split Dataset

In [16]:
# Splite data into training and testing
from sklearn import model_selection

# Reserve 20% for testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

print('training data has %d observation with %d features'% X_train.shape)
print('test data has %d observation with %d features'% X_test.shape)

training data has 5625 observation with 41 features
test data has 1407 observation with 41 features


## 3.2 model training and selection

In [0]:
#@title build models
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn import tree

# Logistic Regression
classifier_logistic = LogisticRegression()

# K Nearest Neighbors
classifier_KNN = KNeighborsClassifier()

# Random Forest
classifier_RF = RandomForestClassifier()
# SVM
classifier_SVC = SVC()

# Decision tree
classifier_DT = tree.DecisionTreeClassifier()


In [18]:
# Use 5-fold Cross Validation to get the accuracy for different models
model_names = ['Logistic Regression','KNN','Random Forest','SVM']
model_list = [classifier_logistic, classifier_KNN, classifier_RF,classifier_SVC]
count = 0

for classifier in model_list:
    cv_score = model_selection.cross_val_score(classifier, X, y, cv=5)
    # cprint(cv_score)
    print('Model accuracy of %s is: %.3f'%(model_names[count],cv_score.mean()))
    count += 1

Model accuracy of Logistic Regression is: 0.803
Model accuracy of KNN is: 0.761
Model accuracy of Random Forest is: 0.782
Model accuracy of SVM is: 0.797


#Part 3.3 hyperparameters tuning


In [0]:
from sklearn.model_selection import GridSearchCV

# helper function for printing out grid search results 
def print_grid_search_metrics(gs):
    print ("Best score: %0.3f" % gs.best_score_)
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [20]:
# Possible hyperparamter options for Logistic Regression Regularization
# Penalty is choosed from L1 or L2
# C is the lambda value(weight) for L1 and L2
parameters = {
    'penalty':('l1', 'l2'), 
    'C':(1, 5, 10)
}
Grid_LR = GridSearchCV(LogisticRegression(),parameters, cv=5)
Grid_LR.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ('l1', 'l2'), 'C': (1, 5, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
# the best hyperparameter combination
print_grid_search_metrics(Grid_LR)

Best score: 0.807
Best parameters set:
	C: 5
	penalty: 'l2'


In [0]:
# best model
best_LR_model = Grid_LR.best_estimator_

In [23]:
# Possible hyperparamter options for Random Forest
# Choose the number of trees
parameters = {
    'n_estimators' : [40,60,80]
}
Grid_RF = GridSearchCV(RandomForestClassifier(),parameters, cv=5)
Grid_RF.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [40, 60, 80]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [24]:
# best number of tress
print_grid_search_metrics(Grid_RF)

Best score: 0.790
Best parameters set:
	n_estimators: 40


In [0]:
# best random forest
best_RF_model = Grid_RF.best_estimator_

## Part 3.4: Model Evaluation - Confusion Matrix (Precision, Recall, Accuracy)

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt

# calculate accuracy, precision and recall
def cal_evaluation(classifier, cm):
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]
    accuracy  = (tp + tn) / (tp + fp + fn + tn + 0.0)
    precision = tp / (tp + fp + 0.0)
    recall = tp / (tp + fn + 0.0)
    print (classifier)
    print ("Accuracy is: %0.3f" % accuracy)
    print ("precision is: %0.3f" % precision)
    print ("recall is: %0.3f" % recall)

# print out confusion matrices
def draw_confusion_matrices(confusion_matricies):
    class_names = ['Not','Churn']
    for cm in confusion_matrices:
        classifier, cm = cm[0], cm[1]
        cal_evaluation(classifier, cm)
        fig = plt.figure()
        ax = fig.add_subplot(111)
        cax = ax.matshow(cm, interpolation='nearest',cmap=plt.get_cmap('Reds'))
        plt.title('Confusion matrix for %s' % classifier)
        fig.colorbar(cax)
        ax.set_xticklabels([''] + class_names)
        ax.set_yticklabels([''] + class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()

In [27]:
%matplotlib inline

# Confusion matrix, accuracy, precison and recall for random forest and logistic regression
confusion_matrices = [
    ("Random Forest", confusion_matrix(y_test,best_RF_model.predict(X_test))),
    ("Logistic Regression", confusion_matrix(y_test,best_LR_model.predict(X_test))),
]

draw_confusion_matrices(confusion_matrices)

Random Forest
Accuracy is: 0.790
precision is: 0.615
recall is: 0.493


Logistic Regression
Accuracy is: 0.796
precision is: 0.619
recall is: 0.543
